### Simple GenAI app using Langchain

In [34]:
import os
from dotenv import load_dotenv
load_dotenv()

## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [37]:
## Data Ingestion--> From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader
import bs4

In [86]:
loader=WebBaseLoader("https://www.enadoc.com/blogs/everything-you-need-to-know-about-enadoc-evia-sign/")
loader

In [87]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.enadoc.com/blogs/everything-you-need-to-know-about-enadoc-evia-sign/', 'title': 'Everything you need to know about Enadoc Evia Sign - Enadoc', 'description': 'Legality, Compliance, Security. Streamline document signing with confidence, ensuring legal validity and data protection.', 'language': 'en-US'}, page_content="   Everything you need to know about Enadoc Evia Sign - Enadoc      Skip to content  Login to Evia Sign              Our Solutions        Digital and E-signature       Enterprise Content Management       Business Process Automation       Outsourced Scanning Services    Industries    Roles    Be a Partner   In business, we use “electronic” and “digital” signatures synonymously. However, this may not be correct for all applications.   LEARN MORE      Products Products    Sign   Automate   Archive   Document Management System (DMS)   Business Process Automation   Outsourced Scanning Services  About Enadoc        Our Purpose       Tes

In [88]:
## Load data--> Docs--> Split into chunk docs--> text--> Vectors-->Vector embeddings--> VectorStoreDB

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [89]:
documents

[Document(metadata={'source': 'https://www.enadoc.com/blogs/everything-you-need-to-know-about-enadoc-evia-sign/', 'title': 'Everything you need to know about Enadoc Evia Sign - Enadoc', 'description': 'Legality, Compliance, Security. Streamline document signing with confidence, ensuring legal validity and data protection.', 'language': 'en-US'}, page_content='Everything you need to know about Enadoc Evia Sign - Enadoc      Skip to content  Login to Evia Sign              Our Solutions        Digital and E-signature       Enterprise Content Management       Business Process Automation       Outsourced Scanning Services    Industries    Roles    Be a Partner   In business, we use “electronic” and “digital” signatures synonymously. However, this may not be correct for all applications.   LEARN MORE      Products Products    Sign   Automate   Archive   Document Management System (DMS)   Business Process Automation   Outsourced Scanning Services  About Enadoc        Our Purpose       Testim

In [90]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=(
    OllamaEmbeddings(model="llama3.1") 
)

In [91]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [92]:
vectorstoredb

In [93]:
## Query from a vectorDB
query="AES requires ID verification. It adds an extra layer"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'and securely deliver it through a pre-defined method — SMS or email. The successful validation of the OTP serves as the second factor of authentication, further confirming the user’s identity and authorizing access to the document and sensitive action, such as placing a digital signature.\xa0The way forward in the legality and compliance lane of Evia Sign\xa0Evia Sign qualifies as an Advanced Electronic Signature (AES) in the three-tiered e-signature types identified by the eIDAS regulation passed by the European Commission in 2014 — Due to its incorporation of the digital certificate from GlobalSign, a trusted third-party source, within the application.\xa0The types of e-signatures and eIDAS requirements:\xa0Simple Electronic Signatures (SES):\xa0These signatures don’t require ID verification from the signatory. They are the most basic form of electronic signatures and provide a low level of assurance.\xa0Advanced Electronic Signatures (AES):\xa0AES requires ID verification. It adds 

In [94]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama3.1")

In [95]:
## Retrieval chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following questions based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| Ollama(model='llama3.1')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [96]:
from langchain_core.documents import Document

document_chain.invoke(
    {
        "input":"AES requires ID verification. It adds an extra layer",
        "context":[Document(page_content="AES requires ID verification. It adds an extra layer of confidence by verifying the identity of the signatory and enhancing the integrity of the electronic document, making it more reliable")]
    }
)

'Here are the answers to your implied questions based on the provided context:\n\n1. Does AES require ID verification? \nAnswer: Yes.\n\n2. What is the purpose of adding an extra layer of confidence in AES? \nAnswer: To verify the identity of the signatory and enhance the integrity of the electronic document.\n\n3. How does AES make the electronic document more reliable? \nAnswer: By verifying the identity of the signatory and enhancing the integrity of the electronic document.'

In [97]:
### Input--> Retriever--> VectorStoreDB
vectorstoredb

In [98]:
retriever=vectorstoredb.as_retriever()

In [99]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [100]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000129A5E2F9D0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | Oll

In [104]:
## Get the response from the LLM
response=retrieval_chain.invoke({"input":"What are Qualified Electronic Signatures (QES)?"})
response["answer"]

"Based on the provided context, here are the answers to the questions:\n\n1. The use of digital signatures eliminates the need for what?\n Answer: **wet ink signatures** or physical signatures.\n\n2. What is achieved by offering flexibility in Evia Sign's digital signature platform?\nAnswer: **More options for authentication while maintaining compliance with legal standards**, allowing signatories to specify whether they can use their personal certificates (such as smart cards) for signing purposes.\n\n3. What benefits does Enadoc's ISO 27001:2022 certification provide?\n Answer: **An extra layer of security** for businesses using Enadoc's digital signature platform, ensuring compliance with international standards for information security.\n\n4. How are documents sealed and guaranteed to be trustworthy in Evia Sign?\nAnswer: **By being sealed with a GlobalSign digital certificate**, which guarantees the integrity of signed documents, making them tamper-evident, trustworthy, and legall

In [105]:
response

{'input': 'What are Qualified Electronic Signatures (QES)?',
 'context': [Document(metadata={'source': 'https://www.enadoc.com/blogs/everything-you-need-to-know-about-enadoc-evia-sign/', 'title': 'Everything you need to know about Enadoc Evia Sign - Enadoc', 'description': 'Legality, Compliance, Security. Streamline document signing with confidence, ensuring legal validity and data protection.', 'language': 'en-US'}, page_content='based\xa0 documents Firstly, the use of digital signatures eliminates the need for […] Learn More  Copyright 2022. Enadoc. All rights reserved.ABOUT USOur PurposeOur ExecutivesCareersSOLUTIONSDigital SignatureEnterprise Content ManagementBusiness Process AutomationDigitization ServicesPRODUCTSEvia SignEvia ArchiveEvia AutomateEnadoc ECMFlowdohOutsourced Scanning ServiceCONTENT HUBEventsBlogsNewsVideosResourcesRoles and IndustriesPLANS AND PRICINGEvia SignBECOME A PARTNEROverview                     Privacy Policy Terms of use About Enadoc content hub Solution

In [106]:
response["context"]

[Document(metadata={'source': 'https://www.enadoc.com/blogs/everything-you-need-to-know-about-enadoc-evia-sign/', 'title': 'Everything you need to know about Enadoc Evia Sign - Enadoc', 'description': 'Legality, Compliance, Security. Streamline document signing with confidence, ensuring legal validity and data protection.', 'language': 'en-US'}, page_content='based\xa0 documents Firstly, the use of digital signatures eliminates the need for […] Learn More  Copyright 2022. Enadoc. All rights reserved.ABOUT USOur PurposeOur ExecutivesCareersSOLUTIONSDigital SignatureEnterprise Content ManagementBusiness Process AutomationDigitization ServicesPRODUCTSEvia SignEvia ArchiveEvia AutomateEnadoc ECMFlowdohOutsourced Scanning ServiceCONTENT HUBEventsBlogsNewsVideosResourcesRoles and IndustriesPLANS AND PRICINGEvia SignBECOME A PARTNEROverview                     Privacy Policy Terms of use About Enadoc content hub Solutions talk to us Products Become a Partner Plans and Pricing                 